In [1]:
from tqdm.notebook import tqdm
import math
import gym
import torch
import torch.optim as optim 
from torch.utils.tensorboard import SummaryWriter
from collections import deque

from active_rl.networks.dqn_atari import ENS_DQN
from active_rl.utils.memory import ReplayMemory, GenericRankedDoubleStatesReplayMemory, GenericLabelledReplayMemory
from active_rl.utils.optimization import AMN_optimization_ensemble
from active_rl.environments.atari_wrappers import make_atari, wrap_deepmind
from active_rl.utils.atari_utils import fp, ActionSelector, evaluate
from active_rl.utils.acquisition_functions import ens_BALD_diff_wrapper

In [2]:
env_name = 'Breakout'
env_raw = make_atari('{}NoFrameskip-v4'.format(env_name))
env = wrap_deepmind(env_raw, frame_stack=False, episode_life=False, clip_rewards=True)
c,h,w = c,h,w = fp(env.reset()).shape
n_actions = env.action_space.n

In [3]:
BATCH_SIZE = 64
LR = 0.0000625
GAMMA = 0.99
EPS = 0.05
TAU=0.1
ALPHA=0.4
NUM_STEPS = 10000000
NOT_LABELLED_CAPACITY = 10000
LABELLED_CAPACITY = 100000
INITIAL_STEPS=NOT_LABELLED_CAPACITY
TRAINING_PER_LABEL = 40.
PERCENTAGE = 0.1
TRAINING_ITER = int(TRAINING_PER_LABEL * NOT_LABELLED_CAPACITY * PERCENTAGE)

NAME = f'AMN_Ensemble_BAALD_diff_ALPHA{ALPHA}'

In [4]:
device = 'cuda:1'
# AMN_net = MC_DQN(n_actions).to(device)
AMN_net = ENS_DQN(n_actions).to(device)
expert_net = torch.load("models/expert_Breakout_step9000000").to(device)
AMN_net.apply(AMN_net.init_weights)
expert_net.eval()
# optimizer = optim.Adam(AMN_net.parameters(), lr=LR, eps=1.5e-4)
optimizer = optim.Adam(AMN_net.parameters(), lr=LR, eps=1.5e-4)

In [5]:
rank_func = ens_BALD_diff_wrapper(AMN_net, TAU, ALPHA, BATCH_SIZE, device)

In [6]:
memory_not_labelled = GenericRankedDoubleStatesReplayMemory(NOT_LABELLED_CAPACITY, [5,h,w], n_actions, rank_func)
memory_labelled = ReplayMemory(LABELLED_CAPACITY, [5,h,w], n_actions, None)
memory = GenericLabelledReplayMemory(memory_not_labelled, memory_labelled)
action_selector = ActionSelector(EPS, EPS, AMN_net, 1, n_actions, device)

In [7]:
steps_done = 0
writer = SummaryWriter(f'runs/{NAME}')

In [8]:
q = deque(maxlen=5)
done=True
eps = 0
episode_len = 0
num_labels = 0

In [ ]:
progressive = tqdm(range(NUM_STEPS), total=NUM_STEPS, ncols=400, leave=False, unit='b')
for step in progressive:
  if done:
    env.reset()
    sum_reward = 0
    episode_len = 0
    img, _, _, _ = env.step(1) # BREAKOUT specific !!!
    for i in range(10): # no-op
      n_frame, _, _, _ = env.step(0)
      n_frame = fp(n_frame)
      q.append(n_frame)
        
  # Select and perform an action
  state = torch.cat(list(q))[1:].unsqueeze(0)
  action, eps = action_selector.select_action(state)
  n_frame, reward, done, info = env.step(action)
  n_frame = fp(n_frame)

  # 5 frame as memory
  q.append(n_frame)
  memory.push(torch.cat(list(q)).unsqueeze(0), action, reward, done) # here the n_frame means next frame from the previous time step
  episode_len += 1

  # Perform one step of the optimization (on the target network)
  if step % NOT_LABELLED_CAPACITY == 0 and step >= INITIAL_STEPS:
    num_labels += memory.label_sample_percentage(PERCENTAGE)
    loss = 0
    for _ in range(TRAINING_ITER):
      loss += AMN_optimization_ensemble(AMN_net, expert_net, optimizer, memory, batch_size=BATCH_SIZE, 
                                        device=device)
    loss /= TRAINING_ITER
    writer.add_scalar('loss_vs_#labels', loss, num_labels)
    writer.add_scalar('loss_vs_steps', loss, step)
    
    evaluated_reward = evaluate(step, AMN_net, device, env_raw, n_actions, eps=0.05, num_episode=15)
    writer.add_scalar('reward_vs_#labels', evaluated_reward, num_labels)